In [144]:
import numpy as np
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from GRU_model import GRUNet
from dataset import SolarDataset
import os

In [145]:
solar_test = np.genfromtxt('data/test_data/test_solar2.csv', delimiter=',', skip_header=1)
wind_test = np.genfromtxt('data/test_data/test_wind2.csv', delimiter=',', skip_header=1)

solar_no_norm = np.genfromtxt('data/test_data/test_solar_unnorm.csv', delimiter=',', skip_header=1)
wind_no_norm = np.genfromtxt('data/test_data/test_wind_unnorm.csv', delimiter=',', skip_header=1)

In [146]:
test_results = 'test_results/'
if not os.path.exists(test_results):
    os.makedirs(test_results)

In [147]:
solar_test[:5]

array([[-1.        , -0.50610107, -0.18026097,  0.79607825,  0.28359466,
        -0.99999934,  0.        ,  1.        ,  5.        ,  1.        ,
         1.        ],
       [-1.        , -0.50946691, -0.18462544,  0.79607825,  0.37282326,
        -0.99999934,  1.        ,  1.        ,  5.        ,  1.        ,
         1.        ],
       [-1.        , -0.52549753, -0.19383917,  0.79607825,  0.38301607,
        -0.99999934,  2.        ,  1.        ,  5.        ,  1.        ,
         1.        ],
       [-1.        , -0.54033759, -0.20387625,  0.79607825,  0.37215552,
        -0.99999934,  3.        ,  1.        ,  5.        ,  1.        ,
         1.        ],
       [-1.        , -0.55918026, -0.21334682,  0.79607825,  0.3607097 ,
        -0.99999934,  4.        ,  1.        ,  5.        ,  1.        ,
         1.        ]])

In [148]:
def move_sliding_window(data, window_size, inputs_cols_indices, label_col_index):
    """
    data: numpy array including data
    window_size: size of window
    inputs_cols_indices: col indices to include
    """

    # (# instances created by movement, seq_len (timestamps), # features (input_len))
    inputs = np.zeros((len(data) - window_size, window_size, len(inputs_cols_indices)))
    labels = np.zeros(len(data) - window_size)

    for i in range(window_size, len(data)):
        inputs[i - window_size] = data[i - window_size : i, inputs_cols_indices]
        labels[i - window_size] = data[i, label_col_index]
    inputs = inputs.reshape(-1, window_size, len(inputs_cols_indices))
    labels = labels.reshape(-1, 1)
    print(inputs.shape, labels.shape)

    return inputs, labels

In [149]:
def move_sliding_window_24h(data, window_size, inputs_cols_indices, label_col_index, forecast_horizon=24):
    """
    data: numpy array including data
    window_size: size of window
    inputs_cols_indices: col indices to include
    label_col_index: index of the label column in data
    forecast_horizon: number of time steps ahead to predict
    """

    # Calculate the number of instances based on the available data minus the window size and forecast horizon
    num_instances = len(data) - window_size - forecast_horizon + 1

    # (# instances created by movement, seq_len (timestamps), # features (input_len))
    inputs = np.zeros((num_instances, window_size, len(inputs_cols_indices)))
    labels = np.zeros(num_instances)

    for i in range(num_instances):
        inputs[i] = data[i:i + window_size, inputs_cols_indices]
        labels[i] = data[i + window_size + forecast_horizon - 1, label_col_index]  # Label is forecast_horizon steps ahead

    inputs = inputs.reshape(-1, window_size, len(inputs_cols_indices))
    labels = labels.reshape(-1, 1)
    print(inputs.shape, labels.shape)
    return inputs, labels

In [150]:
criterion = nn.MSELoss()

In [151]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [152]:
WINDOW_SIZE = 25
solar_X_1h, solar_y_1h = move_sliding_window(solar_test, WINDOW_SIZE, range(11), 0)
dataloader = torch.utils.data.DataLoader(SolarDataset(solar_X_1h, solar_y_1h), batch_size=1, shuffle=False)


(8735, 25, 11) (8735, 1)


In [153]:
print(solar_X_1h[0], solar_y_1h[0])

[[-1.         -0.50610107 -0.18026097  0.79607825  0.28359466 -0.99999934
   0.          1.          5.          1.          1.        ]
 [-1.         -0.50946691 -0.18462544  0.79607825  0.37282326 -0.99999934
   1.          1.          5.          1.          1.        ]
 [-1.         -0.52549753 -0.19383917  0.79607825  0.38301607 -0.99999934
   2.          1.          5.          1.          1.        ]
 [-1.         -0.54033759 -0.20387625  0.79607825  0.37215552 -0.99999934
   3.          1.          5.          1.          1.        ]
 [-1.         -0.55918026 -0.21334682  0.79607825  0.3607097  -0.99999934
   4.          1.          5.          1.          1.        ]
 [-1.         -0.56961327 -0.21979821  0.79607825  0.36697497 -0.99999934
   5.          1.          5.          1.          1.        ]
 [-1.         -0.57983419 -0.22414141  0.79607825  0.36939747 -0.99999934
   6.          1.          5.          1.          1.        ]
 [-0.99997378 -0.59589385 -0.23144801  0.

In [154]:

INPUT_SIZE = solar_X_1h.shape[2]
OUTPUT_SIZE = 1
HIDDEN_SIZE = 64
NUM_LAYERS = 2

In [155]:
def evaluate(model, dataloader, criterion):
    model.eval()
    outputs = []
    targets = []
    losses = []
    start_time = time.process_time()
    # get data of test data for each state
    
    
        # predict outputs
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader):
            inputs = inputs.float().to(device)
            labels = labels.float().to(device)

            h = model.init_hidden(1)
            
            # drop first dimension of h
           
           # print(inputs.shape, h.shape)
            out, h = model(inputs, h)
            
            loss = torch.sqrt(criterion(labels, out) + 1e-8)
            #print(f"Loss: {loss.item()}")
            losses.append(loss.item())
            outputs.append(out.cpu().numpy())
            targets.append(labels.cpu().numpy())

    print(f"Evaluation Time: {time.process_time()-start_time}")
    
    
    # list of of targets/outputs for each state
    return outputs, targets, losses

In [156]:
solar_1h_model = GRUNet(INPUT_SIZE, 64, 1,1).to(device)
solar_1h_model.load_state_dict(torch.load('solar_1h_model.pth', map_location=device))
solar_1h_model.eval()


GRUNet(
  (gru): GRU(11, 64, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

In [157]:
outputs_solar_1, targets_solar_1, rmse_solar_1 = evaluate(solar_1h_model,dataloader, criterion)

Evaluation Time: 49.40625


In [158]:
# get average loss
print(f"Average RMSE: {np.mean(rmse_solar_1)}")

Average RMSE: 0.031703997584664134


In [159]:
# shape outputs and targets to be able to plot
outputs_solar = np.array(outputs_solar_1).reshape(-1,1)
targets_solar = np.array(targets_solar_1).reshape(-1,1)
#save outputs and targets
np.savetxt(test_results +'/outputs_solar_1h.csv', outputs_solar, delimiter=',')
np.savetxt(test_results + '/targets_solar_1h.csv', targets_solar, delimiter=',')

In [160]:
print(len(outputs_solar))

8735


In [161]:
HIDDEN_SIZE = 16
NUM_LAYERS = 1

wind_X_1h, wind_y_1h = move_sliding_window(wind_test, WINDOW_SIZE, range(11), 0)
dataloader = torch.utils.data.DataLoader(SolarDataset(wind_X_1h, wind_y_1h), batch_size=1, shuffle=False)

(8735, 25, 11) (8735, 1)


In [162]:
wind_model_1h = GRUNet(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, OUTPUT_SIZE).to(device)
wind_model_1h.load_state_dict(torch.load('models/total_wind_1h_model (1).pth', map_location=device))
wind_model_1h.eval()

GRUNet(
  (gru): GRU(11, 16, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=16, out_features=1, bias=True)
)

In [163]:
outputs_wind_1, targets_wind_1, rmse = evaluate(wind_model_1h,dataloader, criterion)


Evaluation Time: 45.96875


In [164]:
print(f"Average RMSE: {np.mean(rmse)}")
# 0.01885407991705509

Average RMSE: 0.022428454782464077


In [165]:
outputs_wind_1h = np.array(outputs_wind_1).reshape(-1,1)
targets_wind_1h = np.array(targets_wind_1).reshape(-1,1)
#save the results
np.savetxt(test_results + "/wind_outputs_1h.csv", outputs_wind_1h, delimiter=",")
np.savetxt(test_results + "/wind_targets_1h.csv", targets_wind_1h, delimiter=",")

In [166]:
solar_X_24h, solar_y_24h = move_sliding_window_24h(solar_test, 72, range(11), 0, 24)#
dataloader = torch.utils.data.DataLoader(SolarDataset(solar_X_24h, solar_y_24h), batch_size=1, shuffle=False)


(8665, 72, 11) (8665, 1)


In [167]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [168]:
HIDDEN_SIZE = 64
NUM_LAYERS = 2
solar_24h_model = GRUNet(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE,NUM_LAYERS).to(device)
solar_24h_model.load_state_dict(torch.load('solar_24h_model.pth', map_location=device))
solar_24h_model.eval()


GRUNet(
  (gru): GRU(11, 64, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

In [169]:
outputs_solar_24, targets_solar_24, rmse = evaluate(solar_24h_model,dataloader, criterion)



Evaluation Time: 210.9375


In [170]:
average_rmse = np.mean(rmse)
print(f"Average RMSE: {average_rmse}")
#Average RMSE: 0.23142031165367208  0.20508469305730742

Average RMSE: 0.1531642803510652


In [171]:
outputs_solar_24 = np.array(outputs_solar_24).reshape(-1,1)
targets_solar_24 = np.array(targets_solar_24).reshape(-1,1)
#save the results
np.savetxt(test_results + "/solar_outputs_24h.csv", outputs_solar_24, delimiter=",")
np.savetxt(test_results + "/solar_targets_24h.csv", targets_solar_24, delimiter=",")

In [172]:
wind_X_24h, wind_y_24h = move_sliding_window_24h(wind_test, 72, range(11), 0, 24)
dataloaders = torch.utils.data.DataLoader(SolarDataset(wind_X_24h, wind_y_24h), batch_size=1, shuffle=False)


(8665, 72, 11) (8665, 1)


In [173]:
HIDDEN_SIZE = 64
NUM_LAYERS = 2
wind_model_24h = GRUNet(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE,NUM_LAYERS).to(device)
wind_model_24h.load_state_dict(torch.load('wind_24h_model.pth', map_location=device))

<All keys matched successfully>

In [174]:
outputs_wind_24, targets_wind_24, rmse = evaluate(wind_model_24h,dataloaders, criterion)


Evaluation Time: 212.0625


In [175]:
outputs_wind_24 = np.array(outputs_wind_24).reshape(-1,1)
targets_wind_24 = np.array(targets_wind_24).reshape(-1,1)
#save the results
np.savetxt(test_results + "/outputs_wind_24h.csv", outputs_wind_24, delimiter=",")
np.savetxt(test_results + "/targets_wind_24h.csv", targets_wind_24, delimiter=",")
print(f"Average RMSE: {np.mean(rmse)}")
#Average RMSE: 0.2523170205728958
# 0.24148374354972024

Average RMSE: 0.2929446832060122
